# Inicialización Grader

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

# Archivos de datos

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


Se descomprime el archivo con los datos:

In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# Importaciones

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report


Se leen los archivos train.csv y test.csv en dataframes de pandas y muestra el tamaño del dataset de entrenamiento (n_filas, n_columnas).

También muestra las primeras filas del train para inspeccionar la estructura de los datos.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape)
train.head()


(692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Definir variables del problema

Define el nombre de la variable objetivo: RENDIMIENTO_GLOBAL.

Separa:

X: todas las columnas excepto la target.
y: la columna objetivo.

Identifica:

numeric_cols: columnas numéricas
cat_cols: columnas categóricas

Devuelve las listas numeric_cols y cat_cols para ver qué columnas han caído en cada grupo.

In [ ]:
target = "RENDIMIENTO_GLOBAL"

X = train.drop(columns=[target])
y = train[target]

numeric_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

numeric_cols, cat_cols


(['ID',
  'PERIODO_ACADEMICO',
  'INDICADOR_1',
  'INDICADOR_2',
  'INDICADOR_3',
  'INDICADOR_4'],
 ['E_PRGM_ACADEMICO',
  'E_PRGM_DEPARTAMENTO',
  'E_VALORMATRICULAUNIVERSIDAD',
  'E_HORASSEMANATRABAJA',
  'F_ESTRATOVIVIENDA',
  'F_TIENEINTERNET',
  'F_EDUCACIONPADRE',
  'F_TIENELAVADORA',
  'F_TIENEAUTOMOVIL',
  'E_PRIVADO_LIBERTAD',
  'E_PAGOMATRICULAPROPIO',
  'F_TIENECOMPUTADOR',
  'F_TIENEINTERNET.1',
  'F_EDUCACIONMADRE'])

# Tomar muestra de 100k para validación cruzada
Imprime las dimensiones de la muestra (X_sample, y_sample).Esto es clave para reducir el tiempo de cómputo ya que la selección de modelo se hace sobre una muestra en lugar de todo el dataset.

In [ ]:
X_sample, _, y_sample, _ = train_test_split(
    X, y,
    train_size=100_000,
    stratify=y,
    random_state=42
)

print(X_sample.shape, y_sample.shape)


(100000, 20) (100000,)


# Preprocesado

Acá se define el pipeline de preprocesamiento:

Para columnas numéricas (numeric_cols): se aplica StandardScaler (centrar y escalar).

Para columnas categóricas (cat_cols): aplica OneHotEncoder (codificación one-hot), ignorando categorías desconocidas en test (handle_unknown="ignore").

Todo se encapsula en un ColumnTransformer llamado preprocess, que luego se utilizará dentro de un Pipeline junto con el modelo.

In [ ]:
preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])


Se crea un diccionario models con los modelos que sevan a evaluar:

"LogisticRegression": regresión logística multinomial con:

max_iter=300 para asegurar convergencia.

n_jobs=-1 para usar todos los núcleos disponibles.

"RandomForest": RandomForest con:

n_estimators=250 árboles.

n_jobs=-1 para paralelizar.

random_state=42 para reproducibilidad.

Este diccionario se recorre después para hacer model selection.

In [ ]:
models = {
    "LogisticRegression": LogisticRegression(
        max_iter=300,
        n_jobs=-1,
        multi_class="multinomial"
    ),

    "RandomForest": RandomForestClassifier(
        n_estimators=250,
        n_jobs=-1,
        random_state=42
    )
}


Define un esquema de validación cruzada

StratifiedKFold con n_splits=3 (3 folds estratificados).

shuffle=True y random_state=42 para barajar y reproducir.

También crea un diccionario vacío para almacenar el accuracy medio de cada modelo.

Para cada modelo en models construye un Pipeline con:

"prep": el preprocess definido antes.

"model": el modelo actual (LogisticRegression o RandomForest).

Ejecuta cross_val_score sobre la muestra (X_sample, y_sample) usando accuracy.

Guarda el promedio de las puntuaciones en results[name].

Imprime algo como:
LogisticRegression → 0.72xx
RandomForest → 0.74xx

Esto implementa la selección de modelo basada en validación cruzada sobre la muestra.

In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    pipe = Pipeline([
        ("prep", preprocess),
        ("model", model)
    ])

    scores = cross_val_score(pipe, X_sample, y_sample, cv=cv, scoring="accuracy", n_jobs=-1)
    results[name] = scores.mean()
    print(f"{name} → {scores.mean():.4f}")


LogisticRegression → 0.4226


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomForest → 0.4011


La siguiente celda selecciona el nombre del modelo con mejor accuracy medio en results usando max sobre los valores.

Deja best_model_name como salida de la celda para ver cuál fue el ganador (por ejemplo, "RandomForest").

In [ ]:
best_model_name = max(results, key=results.get)
best_model_name


'LogisticRegression'

Hace un split hold-out sobre todo el dataset (no solo la muestra)donde es 80% entrenamiento, 20% validación (test_size=0.2). Es estratificado por la variable objetivo y recupera el modelo ganador: best_model = models[best_model_name].

También construye un nuevo Pipeline con:

Preprocesamiento (preprocess).

Modelo ganador (best_model).

Y entrena el pipeline (pipe.fit) con X_train, y_train.

Predice sobre el conjunto de validación X_val.

Por último imprime el accuracy global en este conjunto hold-out.classification_report con precisión, recall, f1 por clase.

Esto valida el modelo seleccionado en un conjunto independiente de la validación cruzada.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

best_model = models[best_model_name]

pipe = Pipeline([
    ("prep", preprocess),
    ("model", best_model)
])

pipe.fit(X_train, y_train)

pred = pipe.predict(X_val)

print("Accuracy:", accuracy_score(y_val, pred))
print(classification_report(y_val, pred))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.4272274368231047
              precision    recall  f1-score   support

        alto       0.53      0.63      0.58     35124
        bajo       0.45      0.56      0.50     34597
  medio-alto       0.32      0.26      0.29     34324
  medio-bajo       0.33      0.25      0.29     34455

    accuracy                           0.43    138500
   macro avg       0.41      0.43      0.41    138500
weighted avg       0.41      0.43      0.41    138500



La siguiente celda crea el pipeline final pipe_final con el mismo preprocesamiento y el modelo ganador (best_model).

Entrena con todos los datos de entrenamiento disponibles (X, y) para aprovechar al máximo la información antes de predecir sobre el test.

In [ ]:
pipe_final = Pipeline([
    ("prep", preprocess),
    ("model", best_model)
])

pipe_final.fit(X, y)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['ID', 'PERIODO_ACADEMICO',
                                                   'INDICADOR_1', 'INDICADOR_2',
                                                   'INDICADOR_3',
                                                   'INDICADOR_4']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['E_PRGM_ACADEMICO',
                                                   'E_PRGM_DEPARTAMENTO',
                                                   'E_VALORMATRICULAUNIVERSIDAD',
                                                   'E_HORASSEMANATRABAJA',
                                                   'F_ESTRATOVIVIENDA',
                                                   'F_TIENEINTERNET',
                                                   'F_EDUCACIONPADRE',
                                                   'F_TIENELAVADORA',
                                                   'F_TIENEAUTOMOVIL',
                                                   'E_PRIVADO_LIBERTAD',
                                                   'E_PAGOMATRICULAPROPIO',
                                                   'F_TIENECOMPUTADOR',
                                                   'F_TIENEINTERNET.1',
                                                   'F_EDUCACIONMADRE'])])),
                ('model',
                 LogisticRegression(max_iter=300, multi_class='multinomial',
                                    n_jobs=-1))])

# Predicción sobre test

Usa el pipe_final ya entrenado para predecir las etiquetas de RENDIMIENTO_GLOBAL sobre el dataframe test.

Construye un dataframe submission con columna "ID" tomada del test.

Columna "RENDIMIENTO_GLOBAL" con las predicciones del modelo.

In [ ]:
test_pred = pipe_final.predict(test)

submission = pd.DataFrame({
    "ID": test["ID"],             # ← corregido
    "RENDIMIENTO_GLOBAL": test_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo
